In [ ]:
%cd ../..
%load_ext autoreload

%autoreload 2

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rts.utils import dataframe_from_hdf5
from rts.features.pose import load_all_poses, draw_pose, format_keypoints_to_read
from rts.features.pose import KEYPOINTS_NAMES, ANGLES_ASSOCIATIONS

archive_path = "/mnt/g/ioc/data/"
poses_folder = "data/test_poses/"
sequences_folder = "/mnt/g/ioc/sequences/"

In [ ]:
poses = load_all_poses(poses_folder)
data = dataframe_from_hdf5(archive_path, "metadata")

pose_df = pd.merge(pd.DataFrame(poses), data[["seq_id", "sport"]], left_on="video_name", right_on="seq_id")

# Similar poses

In [ ]:
# Find nearest neighbors

from sklearn.neighbors import NearestNeighbors

def get_nearest_neighbors(input_pose, n_neighbors=5, dist_threshold = 0.05):
    other_poses = pose_df[pose_df["video_name"] != input_pose["video_name"]].reset_index(drop=True)
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine').fit(other_poses["angle_vec"].tolist())
    distances, indices = nbrs.kneighbors([input_pose["angle_vec"]])
    results = other_poses.iloc[indices[0]].reset_index(drop=True)
    results["distance"] = distances[0]
    results = results[results["distance"] < dist_threshold]
    return results

In [ ]:
n_neighbors = 5
for i in range(5):
    input_pose = pose_df.sample(1).iloc[0]
    results = get_nearest_neighbors(input_pose, n_neighbors=n_neighbors)

    fig, axs = plt.subplots(nrows=1, ncols=n_neighbors + 1, figsize=((n_neighbors + 1) * 3, 3))
    axs = axs.flatten()
    draw_pose(input_pose, video_folder=sequences_folder, ax = axs[0], cut = True)
    axs[0].set_title("Input pose")
    for i, pose in results.iterrows():
        draw_pose(pose, video_folder=sequences_folder, ax = axs[i+1], cut = True)
    plt.show()

# Matching Analytics

In [ ]:
def matching_analytics(input_pose, n_neighbors=100, dist_threshold=0.05, show_top_n=5):
    results = get_nearest_neighbors(input_pose, n_neighbors=n_neighbors, dist_threshold=dist_threshold)
    results["sport"].value_counts()

    # Top 5 poses
    fig, axs = plt.subplots(nrows=1, ncols=show_top_n + 1, figsize=((show_top_n + 1) * 3, 3))
    axs = axs.flatten()
    draw_pose(input_pose, video_folder=sequences_folder, ax = axs[0], cut = True)
    axs[0].set_title("Input pose")
    for i, pose in results[:show_top_n].iterrows():
        draw_pose(pose, video_folder=sequences_folder, ax = axs[i+1], cut = True)
    plt.suptitle("Top %d nearest neighbors" % show_top_n, fontsize = 20, y=1.03)
    plt.show()

    # Analytics
    fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(16, 16), gridspec_kw={'width_ratios': [1, 3], 'height_ratios': [1, 2, 2]})
    axs = axs.flatten()

    axs[0].boxplot(results["distance"])
    axs[0].set_title("Distances")

    sports_counts = results["sport"].value_counts()
    axs[1].bar(sports_counts.index, sports_counts.values)
    axs[1].set_title("Sports (n=%d)" % len(results))
    axs[1].set_xticklabels(sports_counts.index, rotation=45)


    input_keypoints_scores = [k[2] for k in input_pose["keypoints"]]
    keypoints_df_scores = pd.DataFrame(results["keypoints"].apply(lambda x: [k[2] for k in x]).tolist(), columns=KEYPOINTS_NAMES)

    axs[2].barh(KEYPOINTS_NAMES, input_keypoints_scores)
    axs[2].set_title("Input keypoints scores")
    keypoints_df_scores.boxplot(column=KEYPOINTS_NAMES, vert=False, ax=axs[3], grid=False)
    axs[3].set_title("Mean keypoints scores of nearest neighbors")

    input_angle_scores = input_pose["angle_score"]
    angle_df_scores = pd.DataFrame(results["angle_score"].tolist(), columns=ANGLES_ASSOCIATIONS.keys())

    axs[4].barh(list(ANGLES_ASSOCIATIONS.keys()), input_angle_scores)
    axs[4].set_title("Input angles scores")
    angle_df_scores.boxplot(column=list(ANGLES_ASSOCIATIONS.keys()), vert=False, ax=axs[5], grid=False)
    axs[5].set_title("Mean angles scores of nearest neighbors")

    plt.suptitle("Nearest neighbors analytics for %s (%s)" % (input_pose["video_name"], input_pose["sport"]), fontsize = 20, y=1.03)
    plt.tight_layout()
    plt.show()

    return results

In [ ]:
input_pose = pose_df.sample(1).iloc[0]

results = matching_analytics(input_pose)

# Analytics by sports

In [ ]:
from collections import Counter

In [ ]:
pose_df = pose_df[pose_df["sport"] != "Non-Sport"]

n_neighbors = 100
nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine').fit(pose_df["angle_vec"].tolist())
distances, indices = nbrs.kneighbors()

pose_df["distance"] = distances.tolist()
pose_df["nbrs_indices"] = indices.tolist()
pose_df["nbrs_sports"] = pose_df["nbrs_indices"].map(lambda x: pose_df.iloc[x]["sport"].value_counts().to_dict())

In [ ]:
sports = pose_df.sport.unique()
pose_df["nbrs_sports"] = pose_df.nbrs_sports.map(lambda x: Counter({sport: x.get(sport, 0) for sport in sports}))
pose_df["nbrs_props"] = pose_df.nbrs_sports.map(lambda x: Counter({k: v / sum(x.values()) for k, v in x.items()}))

In [ ]:
nbrs_sports_counts = pose_df.groupby("sport").nbrs_sports.sum()
nbrs_sports_props = nbrs_sports_counts.apply(lambda x: Counter({sport: x.get(sport, 0) / np.sum(list(x.values())) for sport in sports}))[sports]

In [ ]:
from collections import defaultdict

def merge_and_mean_dicts(dicts):
    result_dict = defaultdict(list)
    
    for d in dicts:
        for key, value in d.items():
            result_dict[key].append(value)
    
    mean_dict = {key: sum(values) / len(values) for key, values in result_dict.items()}
    return mean_dict

nbrs_sports_props = pose_df.groupby("sport").nbrs_props.agg(merge_and_mean_dicts)[sports]

In [ ]:
proportions = np.array(nbrs_sports_props.apply(lambda x: list(x.values())).values.tolist())
labels = nbrs_sports_props.index

plt.figure(figsize=(14, 10))
sns.heatmap(proportions, annot=False, xticklabels=labels, yticklabels=labels, cmap="Blues")
plt.title("Proportion of sports nearest neighbors (top 100)")
plt.show()

# Finding poses specific to each sport

In [ ]:
pose_df["props_same_sport"] = pose_df.apply(lambda df: df["nbrs_props"][df["sport"]], axis=1)

In [ ]:
props_per_sport = pose_df.groupby("sport").props_same_sport.mean()

plt.figure(figsize=(10, 8))
plt.barh(props_per_sport.index, props_per_sport.values)
plt.title("Proportion of same sport nearest neighbors (top 100)")
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
for sport in pose_df.sport.unique():
    props_poses_unique = []
    for t in np.arange(0, 1.0, 0.1):
        sport_df = pose_df[pose_df.sport == sport]
        prop = sport_df[sport_df.props_same_sport > t].shape[0] / sport_df.shape[0]
        props_poses_unique.append(prop)
    plt.plot(np.arange(0, 1.0, 0.1), props_poses_unique, label = sport)

props_poses_unique = []
for t in np.arange(0, 1.0, 0.1):
    prop = pose_df[pose_df.props_same_sport > t].shape[0] / pose_df.shape[0]
    props_poses_unique.append(prop)
plt.plot(np.arange(0, 1.0, 0.1), props_poses_unique, label = "All Sports", ls = "--", color = "black")

plt.title("Proportion of unique poses (top 100)")
plt.xlabel("Threshold")
plt.ylabel("Proportion of nearest poses in the same sport")
plt.legend()
plt.xlim(0,1)
plt.show()

In [ ]:
pose_df[pose_df.props_same_sport > 0.8].sport.value_counts()